https://learn.microsoft.com/en-us/rest/api/authorization/privileged-role-eligibility-rest-sample


In [43]:
import os
import random
from pprint import pprint
from creds import (
    azure_tenant_id,
    azure_client_id,
    azure_client_secret,
    confluence_page_id,
    confluence_token,
    confluence_url,
    confluence_entraid_page_name,
)
from azure.identity import ClientSecretCredential
from azure.mgmt.authorization import AuthorizationManagementClient
from azure.identity import AzureCliCredential

credential = ClientSecretCredential(
    azure_tenant_id, azure_client_id, azure_client_secret
)
tenant_root_id = "5d1a9f9d-201f-4a10-b983-451cf65cbc1e"
tenant_root_id = "8c9f5db3-2e44-4f8b-a262-250d7b24d0f3"
scope = f"/subscriptions/{tenant_root_id}"

client = AuthorizationManagementClient(credential, tenant_root_id)


In [44]:
#assignments = client.role_eligibility_schedules.list_for_scope(scope)
assignments = client.role_eligibility_schedule_instances.list_for_scope(scope)

In [ ]:
results = []
for assignment in assignments:
    #pprint(assignment.__dict__)
    expanded = assignment.expanded_properties
    end_date_time = assignment.end_date_time or "permanent"
    print(expanded.principal.display_name)


    result = {
        "PrincipalName": expanded.principal.display_name,
        "PrincipalEmail": expanded.principal.email,
        "PrincipalType": expanded.principal.type,
        "PrincipalId": expanded.principal.id,
        "RoleName": expanded.role_definition.display_name,
        "RoleType": expanded.role_definition.type,
        "RoleId": expanded.role_definition.id,
        "ScopeId": expanded.scope.id,
        "ScopeName": expanded.scope.display_name,
        "ScopeType": expanded.scope.type,
        "Status": assignment.status,
        "createdOn": assignment.created_on,
        "startDateTime": assignment.start_date_time,
        "endDateTime": end_date_time,
        #"updatedOn": assignment.updated_on,
        "memberType": assignment.member_type,
        "id": assignment.id,
    }

    results.append(result)


In [ ]:
pprint(results)